In [ ]:
# import os

# # import from cmu-tare-model package
# from config import PROJECT_ROOT

# # Measure Package 0: Baseline
# menu_mp = 0
# input_mp = 'baseline'

# print(f"PROJECT_ROOT (from config.py): {PROJECT_ROOT}")

# # Construct the absolute path to the .py file
# relative_path = os.path.join("cmu_tare_model", "model_scenarios", "tare_baseline_v2_1.ipynb")
# file_path = os.path.join(PROJECT_ROOT, relative_path)

# # On Windows, to avoid any path-escape quirks, convert backslashes to forward slashes
# file_path = file_path.replace("\\", "/")

# print(f"Running file: {file_path}")

# # %run magic command to run a .py file and import variables into the current IPython session
# # # If your path has spaces, wrap it in quotes:
# %run -i {file_path} # If your path has NO spaces, no quotes needed.

# print("Baseline Scenario - Model Run Complete")

# # Flag to prevent excessive output in other scenario files
# individual_scenario_run = True

Project root directory: c:\users\14128\research\cmu-tare-model
PROJECT_ROOT (from config.py): c:\users\14128\research\cmu-tare-model
Running file: c:/users/14128/research/cmu-tare-model/cmu_tare_model/model_scenarios/tare_baseline_v2_1.ipynb

Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results



LOAD EUSS DATA FOR BASELINE SCENARIO: MEASURE PACKAGE 0 (MP0)
Load data from the NREL EUSS Database for the baseline scenario and apply various filters.

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, S

# LOAD EUSS DATA: Annual Energy Consumption and Metadata
## MEASURE PACKAGE 7 (MP7): Data for Electric Resistance Cooking

In [2]:
print(f"""
====================================================================================================================================================================
We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.
====================================================================================================================================================================
""")

# Measure Package 7
menu_mp = 7
input_mp = 'upgrade07'

filename = "upgrade07_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp7 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp7.shape}")


# Filter for occupied homes
occupancy_filter = df_euss_am_mp7['in.vacancy_status'] == 'Occupied'
df_euss_am_mp7 = df_euss_am_mp7.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp7.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp7['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp7 = df_euss_am_mp7.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp7.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp7['in.state'].eq(input_state)
    df_euss_am_mp7 = df_euss_am_mp7.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp7['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp7 = df_euss_am_mp7.loc[city_filter]

# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp7.shape}")
print(df_euss_am_mp7)


We assume the use of Electric Resistance (MP7) rather than Induction (MP8).
Electric Resistance is significantly cheaper and only slightly less efficient than Induction.

Retrieved data for filename: upgrade07_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade07_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 346)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 346)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 346)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 346)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            7  242.131013           True   2152.0   
122            7  242.131013           True   2176.0   
1

## MEASURE PACKAGE 8 (MP8): Metadata, Space Heating, Water Heating, and Clothes Drying

In [3]:
# Print debugging information
print_debug = True

if print_debug:
    from cmu_tare_model.utils.create_sample_df import create_sample_df

In [4]:
# Measure Package 8
menu_mp = 8
input_mp = 'upgrade08'
scenario_name = 'Basic-BAU'
cost_scenario = 'BAU Costs'
grid_scenario = 'Current Electricity Grid'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")

filename = "upgrade08_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

# Fix DtypeWarning error in columns 'in.neighbors' and 'in.geometry_stories_low_rise'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_mp8 = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id' (unique identifier)
print(f"DATAFRAME SIZE before applying any filters: {df_euss_am_mp8.shape}")

# Filter for occupied homes
occupancy_filter = df_euss_am_mp8['in.vacancy_status'] == 'Occupied'
df_euss_am_mp8 = df_euss_am_mp8.loc[occupancy_filter]
print(f"DATAFRAME SIZE after filtering for 'Occupied' homes: {df_euss_am_mp8.shape}")

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_mp8['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_mp8 = df_euss_am_mp8.loc[house_type_filter]
print(f"DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: {df_euss_am_mp8.shape}")

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'

# Filter down to state or city
else:
    print(f"You chose to filter for: {input_state}")
    state_filter = df_euss_am_mp8['in.state'].eq(input_state)
    df_euss_am_mp8 = df_euss_am_mp8.loc[state_filter]

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
    # Filter to a city within the selected state
    else:
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")
        city_filter = df_euss_am_mp8['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_mp8 = df_euss_am_mp8.loc[city_filter]

# Display the filtered dataframe
# Display the filtered dataframe
print(f"DATAFRAME SIZE after applying geographic filter: {df_euss_am_mp8.shape}")
print(df_euss_am_mp8)


MODEL SCENARIO
Scenario Basic-BAU:
Basic Retrofit: Measure Package 8
BAU Costs
Current Electricity Grid

Retrieved data for filename: upgrade08_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\upgrade08_metadata_and_annual_results.csv


DATAFRAME SIZE before applying any filters: (548916, 347)
DATAFRAME SIZE after filtering for 'Occupied' homes: (482597, 347)
DATAFRAME SIZE after filtering for 'Single-Family Attached' and 'Single-Family Detached' homes: (331531, 347)
You chose to filter for: PA
You chose to analyze all of state: PA
DATAFRAME SIZE after applying geographic filter: (15651, 347)
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
119            8  242.131013           True   2152.0   
122            8  242.131013           True   2176.0   
150            8  242.131013           True   1690.0   
153        

# Project Future Energy Consumption

In [5]:
from cmu_tare_model.energy_consumption_and_metadata.process_euss_data import df_enduse_compare

print(F"""
====================================================================================================================================================================
LOAD EUSS DATA FOR MEASURE PACKAGE {menu_mp} (MP{menu_mp})
====================================================================================================================================================================
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
====================================================================================================================================================================
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
Documentation for df_enduse_compare function:
{df_enduse_compare.__doc__}
----------------------------------------------------------------

RESULTING DATAFRAME:
""")

# df_enduse_compare(df_mp, category, df_baseline):
df_euss_am_mp8_home = df_enduse_compare(
    df_mp = df_euss_am_mp8,
    input_mp=input_mp,
    menu_mp=menu_mp,
    df_baseline = df_euss_am_baseline_home,
    df_cooking_range=df_euss_am_mp7,
    )



LOAD EUSS DATA FOR MEASURE PACKAGE 8 (MP8)
You'll notice that the number of rows differs from df_euss_am_mp7 and df_euss_am_mp8.
      - df_euss_am_baseline_home has fewer rows (representative dwelling units) because a tech filter was applied. 
      - df_euss_am_mp8_home will have the same number of rows as df_euss_am_baseline_home after df_enduse_compare function is run.
      - df_enduse_compare function performs an inner merge on the two dataframes, keeping only the rows that are present in both dataframes.
df_euss_am_mp8_home will be created by running the df_enduse_compare function (contains post-retrofit consumption data for the entire home in 2024).
process_euss_data.py file contains the function definition.
      
Documentation for df_enduse_compare function:
Creates a comparison DataFrame by merging multiple DataFrames based on measure packages.

    This function constructs a new DataFrame (df_compare) that includes columns
    from df_mp, df_cooking_range, and merges them 

In [7]:
if print_debug:
    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'valid_tech_heating', 'include_heating', 'heating_consumption']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'valid_tech_waterHeating', 'include_waterHeating', 'waterHeating_consumption']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_consumption']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_consumption']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

# PUBLIC IMPACTS: Climate and Health Damages
## Scenarios: No IRA and IRA-Reference

In [8]:
from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import calculate_lifetime_climate_impacts
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import calculate_lifetime_health_impacts

print(f"""
====================================================================================================================================================================
PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS
====================================================================================================================================================================

----------------------------------------------------------------------------------------------
Step 1: Calculate the baseline marginal damages for climate and health-related emissions
----------------------------------------------------------------------------------------------
Done in baseline_v2_1.ipynb

----------------------------------------------------------------------------------------------
Step 2: Calculate the post-retrofit marginal damages for climate and health-related emissions 
----------------------------------------------------------------------------------------------      
Documentation for calculate_lifetime_climate_impacts_sensitivity function:
{calculate_lifetime_climate_impacts.__doc__}
      
Documentation for calculate_lifetime_health_impacts_sensitivity function:
{calculate_lifetime_health_impacts.__doc__}

---------------------------------------------------------------------------------------------
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)
---------------------------------------------------------------------------------------------
Explained and calculated later in the notebook.

""")

# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
# Climate damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_climate = df_euss_am_mp8_home.copy()
df_mp8_IRA_damages_climate = df_euss_am_mp8_home.copy()

# Health damages: No IRA and IRA-Reference
df_mp8_noIRA_damages_health = df_euss_am_mp8_home.copy()
df_mp8_IRA_damages_health = df_euss_am_mp8_home.copy()



PUBLIC IMPACTS: DAMAGES FROM CLIMATE AND HEALTH-RELATED EMISSIONS

----------------------------------------------------------------------------------------------
Step 1: Calculate the baseline marginal damages for climate and health-related emissions
----------------------------------------------------------------------------------------------
Done in baseline_v2_1.ipynb

----------------------------------------------------------------------------------------------
Step 2: Calculate the post-retrofit marginal damages for climate and health-related emissions 
----------------------------------------------------------------------------------------------      
Documentation for calculate_lifetime_climate_impacts_sensitivity function:

    Calculate lifetime climate impacts (CO2e emissions and climate damages) for each
    equipment category across all (mer_type, scc_value) combinations.

    This function processes each equipment category over its lifetime, computing annual
    and lifet

In [9]:
# Health Impacts: Baseline Scenario
print(f"""
====================================================================================================================================================================
df_euss_am_baseline_home: DataFrame containing the baseline scenario data
{df_euss_am_baseline_home}
      
df_baseline_damages_health: DataFrame containing the baseline scenario data with health damages
{df_baseline_damages_health}

""")

if print_debug:
    # df_euss_am_baseline_home
    print(f"Shape of df_euss_am_baseline_home: {df_euss_am_baseline_home.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_euss_am_baseline_home.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)

    # df_baseline_damages_health
    print(f"Shape of df_baseline_damages_health: {df_baseline_damages_health.shape}")

    # Print columns that contain the word "damages"
    damage_columns = [col for col in df_baseline_damages_health.columns if "damages" in col.lower()]
    print("\nColumns containing 'damages':")
    print(damage_columns)


df_euss_am_baseline_home: DataFrame containing the baseline scenario data
         square_footage census_region  census_division census_division_recs  \
bldg_id                                                                       
119              2152.0     Northeast  Middle Atlantic      Middle Atlantic   
122              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
150              1690.0     Northeast  Middle Atlantic      Middle Atlantic   
153              2176.0     Northeast  Middle Atlantic      Middle Atlantic   
162              2663.0     Northeast  Middle Atlantic      Middle Atlantic   
...                 ...           ...              ...                  ...   
549882           1202.0     Northeast  Middle Atlantic      Middle Atlantic   
549915           2176.0     Northeast  Middle Atlantic      Middle Atlantic   
549937            885.0     Northeast  Middle Atlantic      Middle Atlantic   
549963           1690.0     Northeast  Middle Atlantic  

In [10]:
print("""
==================== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=True  # Add this parameter
    )

df_euss_am_mp8_home, df_mp8_noIRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='No Inflation Reduction Act', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=True  # Add this parameter
    )


print("""
==================== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_damages_climate = calculate_lifetime_climate_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_climate,
    verbose=True  # Add this parameter
    )


df_euss_am_mp8_home, df_mp8_IRA_damages_health = calculate_lifetime_health_impacts(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp, 
    policy_scenario='AEO2023 Reference Case', 
    df_baseline_damages=df_baseline_damages_health,
    debug=False,
    verbose=True  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
Post-Retrofit (MP8) Marginal Damages: WHOLE-HOME
Scenario: No Inflation Reduction Act and AEO2023 Reference Case
====================================================================================================================================================================
calculate_emissions_damages.py file contains the definition for the calculate_marginal_damages function.
Additional information on emissions and damage factor lookups can be found in the calculate_emissions_damages.py file as well. 
      
CLIMATE DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Climate Damages (No IRA): df_mp8_noIRA_damages_climate
{df_mp8_noIRA_damages_climate}

Climate Damages (IRA): df_mp8_IRA_damages_climate
{df_mp8_IRA_damages_climate}

HEALTH DAMAGES: No IRA and IRA-Reference
--------------------------------------------------------
Health Damages (No IRA): df_mp8_noIRA_damages_health
{df_mp8_noIRA_damages_health}

Health Damages (IRA): df_mp8_IRA_damages_health
{df_mp8_IRA_damages_health}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}
====================================================================================================================================================================
""")


==================== SCENARIO: No Inflation Reduction Act ==========

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Climate Emissions and Damages from 2024 to 2039 for heating
Measure package calculation for heating:
  - 12266 homes have valid data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Calculating Climate Emissions and Damages from 2024 to 2036 for waterHeating
Measure package calculation for waterHeating:
  - 14999 homes have valid data
  - 15551 homes will receive retrofits
  - 14999 homes have 

In [11]:
if print_debug:
   # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_heating', 'valid_fuel_heating', 'include_heating', 'heating_lifetime_damages']
    )
    print(df_sample_heating)

    # Create a sample dataframe for the waterHeating category
    df_sample_waterHeating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['waterHeating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_damages']
    )
    print(df_sample_waterHeating)

    # Create a sample dataframe for the clothesDrying category
    df_sample_clothesDrying = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['clothesDrying'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_damages']
    )
    print(df_sample_clothesDrying)

    # Create a sample dataframe for the cooking category
    df_sample_cooking = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_damages']
    )
    print(df_sample_cooking)

        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

In [12]:
if print_debug:
    print("""
==================== SUMMARY DATAFRAME WITH LIFETIME DAMAGES ==========
""")
    # Create a sample dataframe for the heating category
    df_main_sample = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_damages']
    )
    print(f"""
df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

{df_main_sample}

-----------------------------------------------------------------------------
""")


    print("""
==================== CLIMATE IMPACTS WITH ANNUAL AND LIFETIME ==========
""")

    # Create a sample dataframe for the heating category
    df_detailed_climate_noIRA = create_sample_df(
        df=df_mp8_noIRA_damages_climate,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_climate_noIRA dataframe is created using df_mp8_noIRA_damages_climate:
-----------------------------------------------------------------------------
          
{df_detailed_climate_noIRA}

-----------------------------------------------------------------------------
""")

    # Create a sample dataframe for the heating category
    df_detailed_climate_IRA = create_sample_df(
        df=df_mp8_IRA_damages_climate,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_climate_IRA dataframe is created using df_mp8_IRA_damages_climate:
-----------------------------------------------------------------------------
          
{df_detailed_climate_IRA}

-----------------------------------------------------------------------------
""")

    print("""
==================== HEALTH IMPACTS WITH ANNUAL AND LIFETIME ==========
""")
    # Create a sample dataframe for the heating category
    df_detailed_health_noIRA = create_sample_df(
        df=df_mp8_noIRA_damages_health,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['preIRA'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_health_noIRA dataframe is created using df_mp8_noIRA_damages_health:
-----------------------------------------------------------------------------
          
{df_detailed_health_noIRA}

-----------------------------------------------------------------------------
""")

    # Create a sample dataframe for the heating category
    df_detailed_health_IRA = create_sample_df(
        df=df_mp8_IRA_damages_health,
        include_groups=['base_equipment'],
        categories=['cooking'],
        scenarios=['iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'cooking_damages_health']
    )
    print(f"""
df_detailed_health_IRA dataframe is created using df_mp8_IRA_damages_health:
-----------------------------------------------------------------------------
          
{df_detailed_health_IRA}

-----------------------------------------------------------------------------
""")


==================== SUMMARY DATAFRAME WITH LIFETIME DAMAGES ==========


df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

        base_cooking_fuel  base_electricity_cooking_consumption  \
bldg_id                                                           
119           Natural Gas                                 32.53   
122           Electricity                                   NaN   
150           Electricity                                   NaN   
153           Electricity                                   NaN   
162           Electricity                                   NaN   
...                   ...                                   ...   
549882        Electricity                                   NaN   
549915        Electricity                                   NaN   
549937        Natural Gas                                 44.25   
549963        Electricity                     

# PRIVATE IMPACTS: FUEL COSTS
## Scenarios: No IRA and IRA-Reference

In [13]:
from cmu_tare_model.private_impact.calculate_lifetime_fuel_costs import calculate_lifetime_fuel_costs

print(f"""
====================================================================================================================================================================
PRIVATE IMPACTS: OVERVIEW
====================================================================================================================================================================
Step 1: Calculate annual operating (fuel) costs
Step 2: Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)

----------------------------------------------------------------------------------------------------------------------
Step 1: Calculate annual operating (fuel) costs
----------------------------------------------------------------------------------------------------------------------

Documentation for calculate_lifetime_fuel_costs function:
{calculate_lifetime_fuel_costs.__doc__}

====================================================================================================================================================================
FUEL COSTS RESULTS: No IRA and IRA-Reference

""")

print("\n", "Creating dataframe to store annual fuel cost calculations ...")
# Annual fuel costs: No IRA and IRA-Reference
df_mp8_fuel_costs_noIRA = df_euss_am_mp8_home.copy()
df_mp8_fuel_costs_IRA = df_euss_am_mp8_home.copy()


PRIVATE IMPACTS: OVERVIEW
Step 1: Calculate annual operating (fuel) costs
Step 2: Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)

----------------------------------------------------------------------------------------------------------------------
Step 1: Calculate annual operating (fuel) costs
----------------------------------------------------------------------------------------------------------------------

Documentation for calculate_lifetime_fuel_costs function:

    Calculate lifetime fuel costs for each equipment category.

    This function processes each equipment category over its lifetime, computing annual
    and lifetime fuel costs. Results are combined into two DataFrames:
    a main summary (df_main) and a detailed annual breakd

In [14]:
print("""
==================== SCENARIO: No Inflation Reduction Act ==========
""")
df_euss_am_mp8_home, df_mp8_noIRA_fuel_costs = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    df_baseline_costs=df_baseline_fuel_costs  # Add this line
    )

print("""
==================== SCENARIO: Inflation Reduction Act (AEO2023 Reference Case) ==========
""")
df_euss_am_mp8_home, df_mp8_IRA_fuel_costs = calculate_lifetime_fuel_costs(
    df=df_euss_am_mp8_home,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    df_baseline_costs=df_baseline_fuel_costs  # Add this line
    )


print(f"""  
====================================================================================================================================================================
Creating Dataframes for Lifetime Fuel Costs ...

FUEL COSTS (No IRA): df_mp8_noIRA_fuel_costs
{df_mp8_noIRA_fuel_costs}

FUEL COSTS (IRA): df_mp8_IRA_fuel_costs
{df_mp8_IRA_fuel_costs}

SUMMARY DATAFRAME FOR MP8: df_euss_am_mp8_home
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")


==================== SCENARIO: No Inflation Reduction Act ==========

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Calculating Fuel Costs from 2024 to 2039 for heating
Measure package calculation for heating:
  - 12266 homes have valid data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Calculating Fuel Costs from 2024 to 2036 for waterHeating
Measure package calculation for waterHeating:
  - 14999 homes have valid data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retro

In [15]:
if print_debug:
    # Create a sample dataframe for the heating category
    df_main_sample = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment', 'costs'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_fuel_cost']
    )
    print(f"""
df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

{df_main_sample}

-----------------------------------------------------------------------------
""")

    # Create a sample dataframe for the heating category
    df_detailed_test = create_sample_df(
        df=df_mp8_IRA_fuel_costs,
        include_groups=['base_equipment', 'costs'],
        categories=['cooking'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', f'_fuel_cost']
    )
    print(f"""
df_detailed_test dataframe is created using df_mp8_IRA_fuel_costs:
-----------------------------------------------------------------------------
          
{df_detailed_test}

-----------------------------------------------------------------------------
""")

    # # Create a sample dataframe for the waterHeating category
    # df_sample_waterHeating = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['waterHeating'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_waterHeating', 'valid_fuel_waterHeating', 'include_waterHeating', 'waterHeating_lifetime_fuel_cost']
    # )
    # print(df_sample_waterHeating)

    # # Create a sample dataframe for the clothesDrying category
    # df_sample_clothesDrying = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['clothesDrying'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_clothesDrying', 'valid_fuel_clothesDrying', 'include_clothesDrying', 'clothesDrying_lifetime_fuel_cost']
    # )
    # print(df_sample_clothesDrying)

    # # Create a sample dataframe for the cooking category
    # df_sample_cooking = create_sample_df(
    #     df=df_euss_am_mp8_home,
    #     include_groups=['base_equipment', 'costs'],
    #     categories=['cooking'],
    #     scenarios=['preIRA', 'iraRef'],
    #     metrics=[],
    #     mp_number=menu_mp,
    #     regex_patterns=['upgrade_cooking', 'valid_fuel_cooking', 'include_cooking', 'cooking_lifetime_fuel_cost']
    # )
    # print(df_sample_cooking)


df_main_sample dataframe is created using df_euss_am_mp8_home:
-----------------------------------------------------------------------------

        base_cooking_fuel  base_electricity_cooking_consumption  \
bldg_id                                                           
119           Natural Gas                                 32.53   
122           Electricity                                   NaN   
150           Electricity                                   NaN   
153           Electricity                                   NaN   
162           Electricity                                   NaN   
...                   ...                                   ...   
549882        Electricity                                   NaN   
549915        Electricity                                   NaN   
549937        Natural Gas                                 44.25   
549963        Electricity                                   NaN   
549989            Propane                            

# PRIVATE IMPACTS: CAPITAL COSTS
## Scenarios: No IRA and IRA-Reference

In [16]:
from cmu_tare_model.utils.inflation_adjustment import *
from cmu_tare_model.private_impact.calculations.calculate_equipment_installation_costs import (
    obtain_heating_system_specs,
    calculate_heating_installation_premium,
    calculate_installation_cost
    )
from cmu_tare_model.private_impact.calculations.calculate_equipment_replacement_costs import (
    calculate_replacement_cost
)

print(f"""
====================================================================================================================================================================
PRIVATE IMPACTS: NET CAPITAL COSTS AND TOTAL CAPITAL COSTS
====================================================================================================================================================================
Completed Steps:
1. Calculate annual operating (fuel) costs

----------------------------------------------------------------------------------------------------------------------
Step 2: Calculate equipment capital costs (For space heating, include ductwork)
----------------------------------------------------------------------------------------------------------------------
Documentation for obtain_heating_system_specs function (HVAC specific):
{obtain_heating_system_specs.__doc__}

Documentation for calculate_heating_installation_premium function (HVAC specific):
{calculate_heating_installation_premium.__doc__}

Documentation for calculate_installation_cost function:
{calculate_installation_cost.__doc__}

----------------------------------------------------------------------------------------------------------------------
Step 3: Calculate replacement cost (replacing existing piece of eqipment with similar technology)
----------------------------------------------------------------------------------------------------------------------
Documentation for calculate_replacement_cost function:
{calculate_replacement_cost.__doc__}

----------------------------------------------------------------------------------------------------------------------
Step 4 (MP9 AND MP10 SPACE HEATING ONLY): Calculate Enclosure Upgrade Costs
----------------------------------------------------------------------------------------------------------------------
FOR MODERATE (MP9) AND ADVANCED (MP10) ONLY:
calculate_enclosure_upgrade_costs.py file contains the definition for the calculate_enclosure_upgrade_costs function.

====================================================================================================================================================================
LIFETIME CAPITAL COSTS RESULTS: No IRA and IRA-Reference (Rebates)

""")


PRIVATE IMPACTS: NET CAPITAL COSTS AND TOTAL CAPITAL COSTS
Completed Steps:
1. Calculate annual operating (fuel) costs

----------------------------------------------------------------------------------------------------------------------
Step 2: Calculate equipment capital costs (For space heating, include ductwork)
----------------------------------------------------------------------------------------------------------------------
Documentation for obtain_heating_system_specs function (HVAC specific):

    Extract and process heating system specifications from input dataframe.
    
    Calculates total heating load and extracts efficiency metrics from raw data. 
    
    Args:
        df (pd.DataFrame): Input dataframe containing heating system data
        
    Returns:
        pd.DataFrame: Updated dataframe with calculated heating system specs
        
    Raises:
        ValueError: If dataframe is missing required columns
    

Documentation for calculate_heating_installation_

In [17]:
# Collect Capital Cost Data for different End-uses
filename = "tare_retrofit_costs_cpi.xlsx"
relative_path = os.path.join("cmu_tare_model", "data", "retrofit_costs", filename)
file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_heating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='heating_costs')
df_waterHeating_retrofit_costs = pd.read_excel(io=file_path, sheet_name='waterHeating_costs')
df_clothesDrying_retrofit_costs = pd.read_excel(io=file_path, sheet_name='clothesDrying_costs')
df_cooking_retrofit_costs = pd.read_excel(io=file_path, sheet_name='cooking_costs')
df_enclosure_retrofit_costs = pd.read_excel(io=file_path, sheet_name='enclosure_upgrade_costs')

Retrieved data for filename: tare_retrofit_costs_cpi.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\retrofit_costs\tare_retrofit_costs_cpi.xlsx




### Space Heating and No Enclosure Upgrade

#### Space Heating Capital Costs

In [18]:
print("""
========================== Capital Costs: Space Heating ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_kBtuh_progressive', 'cost_per_kBtuh_reference', 'cost_per_kBtuh_conservative',
    'otherCost_progressive', 'otherCost_reference', 'otherCost_conservative'
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_heating_retrofit_costs[column] = round((df_heating_retrofit_costs[column] * df_heating_retrofit_costs['cpi_ratio'] * df_heating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_heating_equipment_cost = df_heating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')

# Call the function and obtain equipment specifications
print("Obtaining heating system specs ...")
df_euss_am_mp8_home = obtain_heating_system_specs(df=df_euss_am_mp8_home)

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Heat Pump for Space Heating (No Enclosure Upgrade) ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_heating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='heating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_heating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='heating')

# Call the function and calculate installation premium based on existing housing characteristics
# calculate_heating_installation_premium(df, menu_mp, cpi_ratio_2023_2013)
print("Calculating Space Heating Specific Premiums (Ex: Removing Hydronic Boiler) ...")
df_euss_am_mp8_home = calculate_heating_installation_premium(df=df_euss_am_mp8_home,
                                                             menu_mp=menu_mp,
                                                             cpi_ratio_2023_2013=cpi_ratio_2023_2013)

# Display the df
print(df_euss_am_mp8_home)


========================== Capital Costs: Space Heating ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...

Obtaining heating system specs ...
Calculating Cost of Retrofit Upgrade: Heat Pump for Space Heating (No Enclosure Upgrade) ...
Starting heating installation cost calculation with validation framework
Measure package calculation for heating:
  - 12266 homes have valid data
  - 15649 homes will receive retrofits
  - 12266 homes have both valid data AND will receive retrofits
  - 3385 homes excluded (values will be NaN)
Found 12266 valid homes out of 15651 for heating installation
After tech filtering: 12266 homes remain valid for heating installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'heating'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting heating replacement cost calculation with validation framework
Measure package calculation for heating:
  - 12266 h

In [19]:
if print_debug:
    # Create a sample dataframe for the heating category
    df_sample_heating = create_sample_df(
        df=df_euss_am_mp8_home,
        include_groups=['base_equipment'],
        categories=['heating'],
        scenarios=['preIRA', 'iraRef'],
        metrics=[],
        mp_number=menu_mp,
        regex_patterns=['valid_fuel_heating', 'include_heating', 'baseline_AFUE', 'baseline_SEER', 'baseline_HSPF', 
                        'hvac_heating_efficiency', 'upgrade_hvac_', 'upgrade_heating','ugrade_newInstall_HSPF']
    )
    print(df_sample_heating)


        base_heating_fuel                         heating_type  \
bldg_id                                                          
119           Natural Gas             Natural Gas Fuel Furnace   
122              Fuel Oil                Fuel Oil Fuel Furnace   
150           Electricity                Electricity Baseboard   
153              Fuel Oil                Fuel Oil Fuel Furnace   
162              Fuel Oil                Fuel Oil Fuel Furnace   
...                   ...                                  ...   
549882        Natural Gas  Natural Gas Fuel Wall/Floor Furnace   
549915        Electricity         Electricity Electric Furnace   
549937        Natural Gas             Natural Gas Fuel Furnace   
549963        Natural Gas             Natural Gas Fuel Furnace   
549989           Fuel Oil                Fuel Oil Fuel Furnace   

         base_electricity_heating_consumption  \
bldg_id                                         
119                                      0.

### Water Heating

In [20]:
print("""
========================== Capital Costs: Water Heating ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...
""")

cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
    'cost_per_gallon_progressive', 'cost_per_gallon_reference', 'cost_per_gallon_conservative',
]

# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_waterHeating_retrofit_costs[column] = round((df_waterHeating_retrofit_costs[column] * df_waterHeating_retrofit_costs['cpi_ratio'] * df_waterHeating_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_waterHeating_equipment_cost = df_waterHeating_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_waterHeating_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Heat Pump Water Heater ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_waterHeating_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='waterHeating')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_waterHeating_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='waterHeating')

# Display the df
print(df_euss_am_mp8_home)


========================== Capital Costs: Water Heating ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...

Calculating Cost of Retrofit Upgrade: Electric Heat Pump Water Heater ...
Starting waterHeating installation cost calculation with validation framework
Measure package calculation for waterHeating:
  - 14999 homes have valid data
  - 15551 homes will receive retrofits
  - 14999 homes have both valid data AND will receive retrofits
  - 652 homes excluded (values will be NaN)
Found 14999 valid homes out of 15651 for waterHeating installation
After tech filtering: 14999 homes remain valid for waterHeating installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'waterHeating'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting waterHeating replacement cost calculation with validation framework
Measure package calculation for waterHeating:
  - 14999 homes have valid data

### Clothes Drying

In [21]:
print("""
========================== Capital Costs: Clothes Drying ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ... 
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_clothesDrying_retrofit_costs[column] = round((df_clothesDrying_retrofit_costs[column] * df_clothesDrying_retrofit_costs['cpi_ratio'] * df_clothesDrying_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_clothesDrying_equipment_cost = df_clothesDrying_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_clothesDrying_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Ventless Heat Pump Clothes Dryer ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_clothesDrying_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='clothesDrying')


# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_clothesDrying_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='clothesDrying')

# Display the df
print(df_euss_am_mp8_home)


========================== Capital Costs: Clothes Drying ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ... 

Calculating Cost of Retrofit Upgrade: Ventless Heat Pump Clothes Dryer ...
Starting clothesDrying installation cost calculation with validation framework
Measure package calculation for clothesDrying:
  - 14743 homes have valid data
  - 14743 homes will receive retrofits
  - 14743 homes have both valid data AND will receive retrofits
  - 908 homes excluded (values will be NaN)
Found 14743 valid homes out of 15651 for clothesDrying installation
After tech filtering: 14743 homes remain valid for clothesDrying installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'clothesDrying'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting clothesDrying replacement cost calculation with validation framework
Measure package calculation for clothesDrying:
  - 14743 homes have 

### Cooking

In [22]:
print("""
========================== Capital Costs: Cooking ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...      
""")

# Columns to update
cost_columns = [
    'unitCost_progressive', 'unitCost_reference', 'unitCost_conservative',
]
 
# Update each cost column by multiplying with cpi_ratio and cost_multiplier
for column in cost_columns:
    df_cooking_retrofit_costs[column] = round((df_cooking_retrofit_costs[column] * df_cooking_retrofit_costs['cpi_ratio'] * df_cooking_retrofit_costs['cost_multiplier']), 2)

# Creating a dictionary from the DataFrame
dict_cooking_equipment_cost = df_cooking_retrofit_costs.set_index(['technology', 'efficiency']).to_dict(orient='index')
# dict_cooking_equipment_cost

# calculate_installation_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Retrofit Upgrade: Electric Resistance Range ...")
df_euss_am_mp8_home = calculate_installation_cost(df=df_euss_am_mp8_home,
                                                  cost_dict=dict_cooking_equipment_cost,
                                                  menu_mp=menu_mp,
                                                  end_use='cooking')

# calculate_replacement_cost(df, cost_dict, menu_mp, end_use)
print("Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...")
df_euss_am_mp8_home = calculate_replacement_cost(df=df_euss_am_mp8_home,
                                                 cost_dict=dict_cooking_equipment_cost,
                                                 menu_mp=menu_mp,
                                                 end_use='cooking')

# Display the df
print(df_euss_am_mp8_home)


========================== Capital Costs: Cooking ==========================

Obtaining Capital Cost Data from Retrofit Cost Spreadsheet ...      

Calculating Cost of Retrofit Upgrade: Electric Resistance Range ...
Starting cooking installation cost calculation with validation framework
Measure package calculation for cooking:
  - 7867 homes have valid data
  - 7867 homes will receive retrofits
  - 7867 homes have both valid data AND will receive retrofits
  - 7784 homes excluded (values will be NaN)
Found 7867 valid homes out of 15651 for cooking installation
After tech filtering: 7867 homes remain valid for cooking installation

Verifying masking for all calculated columns:
Masking 1 columns for category 'cooking'
Calculating Cost of Replacing Existing Equipment with Similar Model/Efficiency ...
Starting cooking replacement cost calculation with validation framework
Measure package calculation for cooking:
  - 7867 homes have valid data
  - 7867 homes will receive retrofits
  - 786

 ## Calculate Rebate Amounts (Applicable to IRA-Reference)

In [23]:
from cmu_tare_model.private_impact.data_processing.determine_rebate_eligibility_and_amount import calculate_percent_AMI, calculate_rebateIRA

print(f"""
====================================================================================================================================================================
CALCULATE HOUSEHOLD PERCENT AREA MEDIAN INCOME (%AMI) AND REBATE ELIGIBILITY/AMOUNTS
====================================================================================================================================================================
determine_rebate_eligibility_and_amount.py file contains the function definitions for calculating rebate amounts and determining household %AMI.
process_income_data_for_rebates.py file contains additional information on data sources and procedures used to process data for determine_rebate_eligibility_and_amount.py file.

Documentation for calculate_percent_AMI function:
{calculate_percent_AMI.__doc__}

Documentation for calculate_rebateIRA function:
{calculate_rebateIRA.__doc__}
-------------------------------------------------------

""")

# Determine Percent AMI and Rebate Amounts
# This needs to be done before running the calculate_percent_AMI function
df_euss_am_mp8_home = df_euss_am_mp8_home.copy()

# calculate_percent_AMI(df_results_IRA, df_county_medianIncome):
df_euss_am_mp8_home = calculate_percent_AMI(df_results_IRA=df_euss_am_mp8_home)

print("Calculating rebate amounts for Space Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="heating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Water Heating ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="waterHeating",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Clothes Drying ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="clothesDrying",
                                          menu_mp=menu_mp)

print("Calculating rebate amounts for Cooking ...")
df_euss_am_mp8_home = calculate_rebateIRA(df_results_IRA=df_euss_am_mp8_home,
                                          category="cooking",
                                          menu_mp=menu_mp)

print(f"""
====================================================================================================================================================================
DATAFRAME: df_euss_am_mp8_home AFTER CALCULATING REBATE AMOUNTS
{df_euss_am_mp8_home}

====================================================================================================================================================================
""")


CALCULATE HOUSEHOLD PERCENT AREA MEDIAN INCOME (%AMI) AND REBATE ELIGIBILITY/AMOUNTS
determine_rebate_eligibility_and_amount.py file contains the function definitions for calculating rebate amounts and determining household %AMI.
process_income_data_for_rebates.py file contains additional information on data sources and procedures used to process data for determine_rebate_eligibility_and_amount.py file.

Documentation for calculate_percent_AMI function:

    Calculates the percentage of Area Median Income (AMI) and assigns income level designations.

    This function processes household income data, calculates the percentage relative to 
    Area Median Income, and creates two categorical classifications:
    1. Detailed income level categories (Low, Moderate, Middle-to-Upper Income)
    2. Binary Low-to-Moderate Income (LMI) or Middle-to-Upper Income (MUI) classification for policy analysis

    Args:
        df_results_IRA: Input DataFrame containing income information with columns

# SCENARIO ANALYSIS: 
## "No Inflation Reduction Act" and "AEO2023 Reference Case"
## Public Impact, Private Impact and Adoption Potential (Degree of Adoption Feasibility)

In [24]:
from cmu_tare_model.private_impact.calculate_lifetime_private_impact import calculate_private_npv
from cmu_tare_model.public_impact.calculate_lifetime_public_impact_sensitivity import calculate_public_npv
from cmu_tare_model.adoption_potential.determine_adoption_potential_sensitivity_robust import *

print(f"""  
====================================================================================================================================================================
SCENARIO ANALYSIS: OVERVIEW
====================================================================================================================================================================

-------------------------------------------------------
PUBLIC IMPACTS: CLIMATE AND HEALTH
-------------------------------------------------------
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the public_impact folder. 
      
Documentation for the calculate_public_NPV function:
{calculate_public_npv.__doc__} 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------
PRIVATE IMPACTS: LIFETIME CAPITAL COSTS AND SAVINGS
--------------------------------------------------------
calculate_lifetime_private_impact.py file contains the definition for the calculate_private_npv function.
Additional information on fuel price lookups as well as capital costs calculation methods can be found in the private_impact folder.

Documentation for the calculate_private_npv function:
{calculate_private_npv.__doc__}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------
ADOPTION POTENTIAL
--------------------------------------------------------
determine_adoption_potential_sensitivity_robust.py file contains the definitions for:
- adoption_decision function (combined climate + health analysis)
- calculate_climate_only_adoption_robust function (climate-only analysis)
- calculate_health_only_adoption_robust function (health-only analysis)
      
Documentation for the adoption_decision function:
{adoption_decision.__doc__}

Documentation for the calculate_climate_only_adoption_robust function:
{calculate_climate_only_adoption_robust.__doc__}

Documentation for the calculate_health_only_adoption_robust function:
{calculate_health_only_adoption_robust.__doc__}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

""")

  
SCENARIO ANALYSIS: OVERVIEW

-------------------------------------------------------
PUBLIC IMPACTS: CLIMATE AND HEALTH
-------------------------------------------------------
calculate_lifetime_public_impact.py file contains the definition for the calculate_public_npv function.
Additional information on emissions/damage factor lookups as well as marginal damages calculation methods can be found in the public_impact folder. 
      
Documentation for the calculate_public_NPV function:

    Calculate the public Net Present Value (NPV) for specific categories of damages,
    considering different policy scenarios related to grid decarbonization.

    The function compares baseline damages with post-measure package (mp) damages
    to determine the avoided damages (benefits) from implementing retrofits.

    This function follows the five-step validation framework:
    1. Mask Initialization: Identifies valid homes using inclusion flags and retrofit status
    2. Series Initialization: 

## BASIC MEASURE PACKAGE (MP8): NO INFLATION REDUCTION ACT
## Assumptions
- NREL End-Use Savings Shapes Database: Measure Package 8
- AEO2023 No Inflation Reduction Act:  HDD and Fuel Price Projections
- Cambium 2021 MidCase Scenario

In [25]:
# Measure Package 8
scenario_name = 'No Inflation Reduction Act'
cost_scenario = 'Fuel Costs: AEO2023 No Inflation Reduction Act'
grid_scenario = 'Electricity Grid: Cambium 2021 MidCase Scenario'

print(f"""
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")


Scenario No Inflation Reduction Act:
Basic Retrofit: Measure Package 8
Fuel Costs: AEO2023 No Inflation Reduction Act
Electricity Grid: Cambium 2021 MidCase Scenario



In [26]:
print(f"""  
====================================================================================================================================================================
SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): PUBLIC IMPACT 
====================================================================================================================================================================
Completed Steps:
1. Calculate the baseline marginal damages for climate and health-related emissions
2. Calculate the post-retrofit marginal damages for climate and health-related emissions

---------------------------------------------------------------------------------------------
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)
---------------------------------------------------------------------------------------------

RESULTS OUTPUT:
""")

# Create copies to prevent overwriting the original dataframe and compare the differences
df_euss_am_mp8_home_ap2 = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_easiur = df_euss_am_mp8_home.copy()
df_euss_am_mp8_home_inmap = df_euss_am_mp8_home.copy()

# ========================== AP2  ========================== 
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='ap2',
    base_year=2024,
    discounting_method='public'
)

# ====================== EASIUR ========================== 
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='easiur',
    base_year=2024,
    discounting_method='public'
)

# ========================== InMAP ========================== 
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_noIRA_damages_climate,
    df_mp_health=df_mp8_noIRA_damages_health,
    menu_mp="8",
    policy_scenario='No Inflation Reduction Act',
    rcm_model='inmap',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV (NO IRA): df_euss_am_mp8_home

-----------------------------------------------
AP2: 
-----------------------------------------------

{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")

  
SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): PUBLIC IMPACT 
Completed Steps:
1. Calculate the baseline marginal damages for climate and health-related emissions
2. Calculate the post-retrofit marginal damages for climate and health-related emissions

---------------------------------------------------------------------------------------------
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)
---------------------------------------------------------------------------------------------

RESULTS OUTPUT:

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              
Baseline Climate Data

In [27]:
print(f"""
====================================================================================================================================================================
SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): PRIVATE IMPACT
====================================================================================================================================================================
Completed Steps:
1. Calculate annual operating (fuel) costs
2. Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
3. Calculate replacement cost (replacing existing piece of eqipment with similar technology)

----------------------------------------------------------------------------------------------------------------------
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)
----------------------------------------------------------------------------------------------------------------------

RESULTS OUTPUT (NO INFLATION REDUCTION ACT):
""")


# ========================== AP2  ========================== 
df_euss_am_mp8_home_ap2 = calculate_private_npv(
    df=df_euss_am_mp8_home_ap2,
    df_fuel_costs=df_mp8_noIRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ====================== EASIUR ========================== 
df_euss_am_mp8_home_easiur = calculate_private_npv(
    df=df_euss_am_mp8_home_easiur,
    df_fuel_costs=df_mp8_noIRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ====================== InMAP ========================== 
df_euss_am_mp8_home_inmap = calculate_private_npv(
    df=df_euss_am_mp8_home_inmap,
    df_fuel_costs=df_mp8_noIRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='No Inflation Reduction Act',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )


print(f"""  
====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV (NO INFLATION REDUCTION ACT): df_euss_am_mp8_home

-----------------------------------------------
AP2: 
-----------------------------------------------

{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")


SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): PRIVATE IMPACT
Completed Steps:
1. Calculate annual operating (fuel) costs
2. Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
3. Calculate replacement cost (replacing existing piece of eqipment with similar technology)

----------------------------------------------------------------------------------------------------------------------
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)
----------------------------------------------------------------------------------------------------------------------

RESULTS OUTPUT (NO INFLATION REDUCTION ACT):

-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_ele

In [28]:
print(f"""
====================================================================================================================================================================
SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): ADOPTION POTENTIAL
====================================================================================================================================================================
determine_adoption_potential_sensitivity_robust.py file contains the definitions for:
- adoption_decision function (combined climate + health analysis)
- calculate_climate_only_adoption_robust function (climate-only analysis)
- calculate_health_only_adoption_robust function (health-only analysis)
      
RESULTS OUTPUT (NO INFLATION REDUCTION ACT):
""")

# ========================== AP2  ========================== 
print("Processing AP2 model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='ap2',
    cr_function='acs',
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='ap2',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_ap2 = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_ap2 = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='ap2',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_ap2 = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='ap2',
    cr_function='h6c',
    verbose=True
    )

# ====================== EASIUR ========================== 
print("Processing EASIUR model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='easiur',
    cr_function='acs',
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='easiur',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_easiur = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_easiur = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='easiur',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_easiur = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='easiur',
    cr_function='h6c',
    verbose=True
    )

# ====================== InMAP ========================== 
print("Processing InMAP model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='inmap',
    cr_function='acs',
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='inmap',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_inmap = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_inmap = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='inmap',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_inmap = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='No Inflation Reduction Act',
    rcm_model='inmap',
    cr_function='h6c',
    verbose=True
    )

print(f"""
====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY (NO INFLATION REDUCTION ACT): df_euss_am_mp8_home
Includes: Combined (Climate + Health), Climate-Only, and Health-Only Adoption Analysis

-----------------------------------------------
AP2: 
-----------------------------------------------

{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")


SCENARIO ANALYSIS (NO INFLATION REDUCTION ACT): ADOPTION POTENTIAL
determine_adoption_potential_sensitivity_robust.py file contains the definitions for:
- adoption_decision function (combined climate + health analysis)
- calculate_climate_only_adoption_robust function (climate-only analysis)
- calculate_health_only_adoption_robust function (health-only analysis)
      
RESULTS OUTPUT (NO INFLATION REDUCTION ACT):

Processing AP2 model...
Fixed 12 duplicate columns
-- Scenario: No Inflation Reduction Act -- 
              scenario_prefix: f'preIRA_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_preIRA'
              

Adoption Analysis: No Inflation Reduction Act | ap2-acs

Verifying masking for all calculated columns:

Summ

## BASIC MEASURE PACKAGE (MP8): AEO2023 REFERENCE CASE
## Assumptions
- NREL End-Use Savings Shapes Database: Measure Package 8
- AEO2023 REFERENCE CASE: HDD and Fuel Price Projections
- Cambium 2023 MidCase Scenario

In [29]:
# Measure Package 8
scenario_name = 'Basic IRA-Reference'
cost_scenario = 'Fuel Costs: AEO2023 Reference Case'
grid_scenario = 'Electricity Grid: Cambium 2023 MidCase Scenario'

print(f"""
====================================================================================================================================================================
MODEL SCENARIO
====================================================================================================================================================================
Scenario {scenario_name}:
Basic Retrofit: Measure Package {menu_mp}
{cost_scenario}
{grid_scenario}
====================================================================================================================================================================
""")


MODEL SCENARIO
Scenario Basic IRA-Reference:
Basic Retrofit: Measure Package 8
Fuel Costs: AEO2023 Reference Case
Electricity Grid: Cambium 2023 MidCase Scenario



In [30]:
print(f"""  
====================================================================================================================================================================
SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): PUBLIC IMPACT 
====================================================================================================================================================================
Completed Steps:
1. Calculate the baseline marginal damages for climate and health-related emissions
2. Calculate the post-retrofit marginal damages for climate and health-related emissions

---------------------------------------------------------------------------------------------
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)
---------------------------------------------------------------------------------------------

RESULTS OUTPUT (AEO2023 REFERENCE CASE):
""")

# ========================== AP2  ========================== 
df_euss_am_mp8_home_ap2 = calculate_public_npv(
    df=df_euss_am_mp8_home_ap2,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='ap2',
    base_year=2024,
    discounting_method='public'
)

# ====================== EASIUR ========================== 
df_euss_am_mp8_home_easiur = calculate_public_npv(
    df=df_euss_am_mp8_home_easiur,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='easiur',
    base_year=2024,
    discounting_method='public'
)

# ========================== InMAP ========================== 
df_euss_am_mp8_home_inmap = calculate_public_npv(
    df=df_euss_am_mp8_home_inmap,
    df_baseline_climate=df_baseline_damages_climate,
    df_baseline_health=df_baseline_damages_health,
    df_mp_climate=df_mp8_IRA_damages_climate,
    df_mp_health=df_mp8_IRA_damages_health,
    menu_mp="8",
    policy_scenario='AEO2023 Reference Case',
    rcm_model='inmap',
    base_year=2024,
    discounting_method='public'
)

print(f"""  
=====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER CALCULATING PUBLIC NPV (AEO2023 REFERENCE CASE): df_euss_am_mp8_home

-----------------------------------------------
AP2: 
-----------------------------------------------
      
{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")

  
SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): PUBLIC IMPACT 
Completed Steps:
1. Calculate the baseline marginal damages for climate and health-related emissions
2. Calculate the post-retrofit marginal damages for climate and health-related emissions

---------------------------------------------------------------------------------------------
Step 3: Discount climate and health impacts and calculate lifetime public impacts (public NPV)
---------------------------------------------------------------------------------------------

RESULTS OUTPUT (AEO2023 REFERENCE CASE):

-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
      

In [31]:
print(f"""
====================================================================================================================================================================
SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): PRIVATE IMPACT
====================================================================================================================================================================
Completed Steps:
1. Calculate annual operating (fuel) costs
2. Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
3. Calculate replacement cost (replacing existing piece of eqipment with similar technology)

----------------------------------------------------------------------------------------------------------------------
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)
----------------------------------------------------------------------------------------------------------------------

RESULTS OUTPUT (AEO2023 REFERENCE CASE):
""")


# ========================== AP2  ========================== 
df_euss_am_mp8_home_ap2 = calculate_private_npv(
    df=df_euss_am_mp8_home_ap2,
    df_fuel_costs=df_mp8_IRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ====================== EASIUR ========================== 
df_euss_am_mp8_home_easiur = calculate_private_npv(
    df=df_euss_am_mp8_home_easiur,
    df_fuel_costs=df_mp8_IRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )

# ====================== InMAP ========================== 
df_euss_am_mp8_home_inmap = calculate_private_npv(
    df=df_euss_am_mp8_home_inmap,
    df_fuel_costs=df_mp8_IRA_fuel_costs,
    df_baseline_costs=df_baseline_fuel_costs,
    menu_mp=menu_mp,
    input_mp=input_mp,
    policy_scenario='AEO2023 Reference Case',
    discounting_method='private_fixed',
    base_year=2024,
    verbose=True  # Add this parameter
    )



print(f"""  
=====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER CALCULATING PRIVATE NPV (AEO2023 REFERENCE CASE): df_euss_am_mp8_home

-----------------------------------------------
AP2: 
-----------------------------------------------
      
{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")


SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): PRIVATE IMPACT
Completed Steps:
1. Calculate annual operating (fuel) costs
2. Calculate equipment capital costs (For space heating, include ductwork and weatherization (MP9 and MP10))
3. Calculate replacement cost (replacing existing piece of eqipment with similar technology)

----------------------------------------------------------------------------------------------------------------------
Step 4: Calculate net equipment capital costs and private NPV (less WTP and more WTP)
----------------------------------------------------------------------------------------------------------------------

RESULTS OUTPUT (AEO2023 REFERENCE CASE):

-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_el

In [32]:
print(f"""
====================================================================================================================================================================
SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): ADOPTION POTENTIAL
====================================================================================================================================================================
determine_adoption_potential_sensitivity_robust.py file contains the definitions for:
- adoption_decision function (combined climate + health analysis)
- calculate_climate_only_adoption_robust function (climate-only analysis)
- calculate_health_only_adoption_robust function (health-only analysis)
      
RESULTS OUTPUT (AEO2023 REFERENCE CASE):
""")

# ========================== AP2  ========================== 
print("Processing AP2 model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='ap2',
    cr_function='acs',
    )

df_euss_am_mp8_home_ap2 = adoption_decision(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='ap2',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_ap2 = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_ap2 = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='ap2',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_ap2 = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_ap2,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='ap2',
    cr_function='h6c',
    verbose=True
    )

# ====================== EASIUR ========================== 
print("Processing EASIUR model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='easiur',
    cr_function='acs',
    )

df_euss_am_mp8_home_easiur = adoption_decision(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='easiur',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_easiur = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_easiur = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='easiur',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_easiur = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_easiur,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='easiur',
    cr_function='h6c',
    verbose=True
    )

# ====================== InMAP ========================== 
print("Processing InMAP model...")

# Combined climate + health analysis (existing)
df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='inmap',
    cr_function='acs',
    )

df_euss_am_mp8_home_inmap = adoption_decision(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='inmap',
    cr_function='h6c',
    )

# Climate-only analysis (new)
df_euss_am_mp8_home_inmap = calculate_climate_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    verbose=True
    )

# Health-only analysis (new)
df_euss_am_mp8_home_inmap = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='inmap',
    cr_function='acs',
    verbose=True
    )

df_euss_am_mp8_home_inmap = calculate_health_only_adoption_robust(
    df=df_euss_am_mp8_home_inmap,
    menu_mp=menu_mp,
    policy_scenario='AEO2023 Reference Case',
    rcm_model='inmap',
    cr_function='h6c',
    verbose=True
    )

print(f"""
====================================================================================================================================================================
DATAFRAME FOR MP8 AFTER DETERMINING ADOPTION FEASIBILITY (AEO2023 REFERENCE CASE): df_euss_am_mp8_home
Includes: Combined (Climate + Health), Climate-Only, and Health-Only Adoption Analysis

-----------------------------------------------   
AP2: 
-----------------------------------------------
      
{df_euss_am_mp8_home_ap2}

-----------------------------------------------
EASIUR:
-----------------------------------------------

{df_euss_am_mp8_home_easiur}

-----------------------------------------------
InMAP:
-----------------------------------------------

{df_euss_am_mp8_home_inmap}
      
""")


SCENARIO ANALYSIS (AEO2023 REFERENCE CASE): ADOPTION POTENTIAL
determine_adoption_potential_sensitivity_robust.py file contains the definitions for:
- adoption_decision function (combined climate + health analysis)
- calculate_climate_only_adoption_robust function (climate-only analysis)
- calculate_health_only_adoption_robust function (health-only analysis)
      
RESULTS OUTPUT (AEO2023 REFERENCE CASE):

Processing AP2 model...
Fixed 12 duplicate columns
-- Scenario: Inflation Reduction Act (IRA) Reference -- 
              scenario_prefix: 'iraRef_mp8_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_IRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health', lookup_fuel_prices: 'lookup_fuel_prices_iraRef'
              

Adoption Analysis: AEO2023 Reference Case | ap2-acs

Verifying masking for all calculated columns:

Summary

In [ ]:
# print(f"""
# =======================================================================================================================================================
# Columns in the df_euss_am_mp8_home_ap2 dataframe:
# =======================================================================================================================================================
# {df_euss_am_mp8_home_ap2.columns.to_list()}""")  # Ensure the columns are loaded correctly for the next steps

# print(f"""
# =======================================================================================================================================================
# Columns in the df_euss_am_mp8_home_easiur dataframe:
# =======================================================================================================================================================
# {df_euss_am_mp8_home_easiur.columns.to_list()}""")  # Ensure the columns are loaded correctly for the next steps

# print(f"""
# =======================================================================================================================================================
# Columns in the df_euss_am_mp8_home_inmap dataframe:
# =======================================================================================================================================================
# {df_euss_am_mp8_home_inmap.columns.to_list()}""")  # Ensure the columns are loaded correctly for the next steps


Columns in the df_euss_am_mp8_home_ap2 dataframe:
['square_footage', 'census_region', 'census_division', 'census_division_recs', 'building_america_climate_zone', 'reeds_balancing_area', 'gea_region', 'state', 'city', 'county', 'county_fips', 'puma', 'county_and_puma', 'weather_file_city', 'Longitude', 'Latitude', 'building_type', 'income', 'federal_poverty_level', 'occupancy', 'tenure', 'vacancy_status', 'base_heating_fuel', 'heating_type', 'hvac_cooling_type', 'vintage', 'base_heating_efficiency', 'base_electricity_heating_consumption', 'base_fuelOil_heating_consumption', 'base_naturalGas_heating_consumption', 'base_propane_heating_consumption', 'base_waterHeating_fuel', 'waterHeating_type', 'base_electricity_waterHeating_consumption', 'base_fuelOil_waterHeating_consumption', 'base_naturalGas_waterHeating_consumption', 'base_propane_waterHeating_consumption', 'base_clothesDrying_fuel', 'base_electricity_clothesDrying_consumption', 'base_naturalGas_clothesDrying_consumption', 'base_pr

# Model Runtime

In [ ]:
# Flag to prevent excessive output in other scenario files
individual_scenario_run = False

# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 4 minutes and 22 seconds to execute.
